In [ ]:
# !pip install openpyxl
# !pip install openpyxl --upgrade
# #!pip install tables

In [1]:
from seeq import spy
import pandas as pd
from seeq.spy.assets import Asset, ItemGroup
import logging
from seeq.spy.assets import Asset, ItemGroup
from importlib import reload

# import numpy as np

# clear log file
file_to_delete = open("log.txt", "w")
file_to_delete.close()
# initialize loging
logging.basicConfig(filename="log.txt", level=logging.INFO)

pd.set_option(
    "display.max_rows",
    None,  # display all rows
    "display.max_columns",
    None,  # display all columns
    "display.max_colwidth",
    None,  # expand column width
    "display.html.use_mathjax",
    False,
)  # disable Latex style mathjax rendering


In [2]:
def uniquetags(rows):
    # get unique list of tags
    tags = set(rows["DpenTag"])
    if "IndependentTag" in rows:
        tags.update(rows["IndependentTag"])

    listtags = list(tags)

    return listtags


def datasearch(datasource, tagstosearch):
    print("Starting Seeq search")
    # get data from seeq

    my_items = pd.DataFrame(
        {"Name": tagstosearch, "Datasource Name": datasource, "Type": "Signal"}
    )
    itemssearch = spy.search(my_items)

    return itemssearch


In [2]:
# ************main Code************
inputdoc_zscore = "InputFildsf-sdfsdf.xlsx"


# ************Initiate new data and perform spy.search************
data_zscore = RawData(inputdoc_zscore)


if data_zscore.spy_search_result.shape[0] > 0:
    # ************Build dataframe for asset builder to work from************
    Optionsdic_zscore = {}
    assets_zscore = assetBuilder_DF(
        Optionsdic_zscore, data_zscore.spy_search_result, data_zscore.rawdata_rows
    )
    assets_zscore.tests_zscore()

    # ************initalize asset builder function************
    # builder_zscore = assetbuilderfunc_zscore(assets_z_score.options_dic, 'Assetbuilder')
    # builder_stagnet = assetbuilderfunc_stagnat(assets_stagnat.options_dic, 'Assetbuilder')


# data_stagnat = RawData(inputdoc_stagnate)
# Optionsdic_stagnat={}
# assets_stagnat = assetBuilder_DF(Optionsdic_stagnat, data_stagnat.spy_search_result, data_stagnat.rawdata_rows)
# assets_stagnat.tests_stagnat()


In [ ]:
# ************build asset tree with asset builder function and class************
spybuilder_zscore = spy.assets.build(builder_zscore, assets_zscore.builderDF)
# spybuilder_stagnet = spy.assets.build(builder_stagnet, assets_stagnat.builderDF)


In [ ]:
# ************push built asset tree to workbook************
spy.push(metadata=spybuilder_zscore, workbook=assets_zscore.options_dic["workbookpath"])
# spy.push(metadata=spybuilder_stagnet, workbook=assets_stagnat.options_dic['workbookpath'])


In [3]:
class RawData:
    def __init__(self, input_file):
        # dictionary to hold datasource and list of tags for that datasource
        self.rawdata_pi_tags = {}

        # List to hold Unique tags to run spy.search on
        self.Unique_tags_tosearch = []
        # DF to hold all spy.search items
        self.spy_search_result = pd.DataFrame()
        # Read Data from excel
        self.rawdata_rows = pd.read_excel(
            input_file, index_col=None, skiprows=0, header=0
        )

        # Get list of data for CC and for PB sites (diff server for Seeq Search) Some tags are same at both sitees
        # and messes up spy.search
        self.pbrows = self.rawdata_rows.loc[
            self.rawdata_rows["servername"].str.contains("FHR PB")
        ]
        self.ccrows = self.rawdata_rows.loc[
            self.rawdata_rows["servername"].str.contains("FHR CC")
        ]
        self.pipelinerows = self.rawdata_rows.loc[
            self.rawdata_rows["servername"].str.contains("Pipelines")
        ]

        self.rawdata_pi_tags = {
            "asdfser": self.pbrows,
            "werwer": self.ccrows,
            "dghfgdshgf": self.pipelinerows,
        }

        # loop through each site and get unique tags then perform spy.search for data
        for key in self.rawdata_pi_tags:
            if len(self.rawdata_pi_tags[key]) > 0:
                # get unique list of tags
                try:
                    tags = set(self.rawdata_pi_tags[key]["DpenTag"])
                    self.Unique_tags_tosearch = list(tags)
                except:
                    logging.info(f"No DpenRow")

                try:
                    tags = set(self.rawdata_pi_tags[key]["IndependentTag"])
                    self.Unique_tags_tosearch = list(tags)
                except:
                    logging.info(f"No Independent row")

                logging.info(f"\n\nStarting -------{key}------- search\n\n")
                self.spy_search_result = datasearch(key, self.Unique_tags_tosearch)
                logging.info(
                    f"NO shearch file exists, create pickle search file for {key}"
                )


In [7]:
class assetBuilder_DF:
    def __init__(self, userOptionsDic, spySearchResultData, original_data):

        # Data Frame used by asset builder
        self.builderDF = pd.DataFrame()
        self.builderDF = spySearchResultData

        # options used by asset builder
        self.options_dic = {}
        # data from spy.search for given input tags
        self.spySearchResultData = spySearchResultData
        # orginal data from input sheet
        self.rowsdf = original_data

    def tests_zscore(self):
        try:
            self.options_dic["workbookpath"] = userOptionsDic["workbookpath"]
        except:
            self.options_dic["workbookpath"] = "MQI >> Test_zscore"
        try:
            self.options_dic["AssetTreeName"] = userOptionsDic["AssetTreeName"]
        except:
            self.options_dic["AssetTreeName"] = "My Unit"
        try:
            self.options_dic["sigma_window"] = userOptionsDic["sigma_window"]
        except:
            self.options_dic["sigma_window"] = "5days"
        try:
            self.options_dic["sigma_freq"] = userOptionsDic["sigma_freq"]
        except:
            self.options_dic["sigma_freq"] = "5days"
        try:
            self.options_dic["maxalert_upper"] = userOptionsDic["maxalert_upper"]
        except:
            self.options_dic["maxalert_upper"] = "20"
        try:
            self.options_dic["maxalert_lower"] = userOptionsDic["maxalert_lower"]
        except:
            self.options_dic["maxalert_lower"] = "-20"
        try:
            self.options_dic["zscore_reqalerttime"] = userOptionsDic[
                "zscore_reqalerttime"
            ]
        except:
            self.options_dic["zscore_reqalerttime"] = "3d"
        try:
            self.options_dic["maxalert_upper_combined"] = userOptionsDic[
                "maxalert_upper_combined"
            ]
        except:
            self.options_dic["maxalert_upper_combined"] = "40"
        try:
            self.options_dic["maxalert_lower_combined"] = userOptionsDic[
                "maxalert_lower_combined"
            ]
        except:
            self.options_dic["maxalert_lower_combined"] = "-40"
        try:
            self.options_dic["maxalert_lower_modified"] = userOptionsDic[
                "maxalert_lower_modified"
            ]
        except:
            self.options_dic["maxalert_lower_modified"] = "-40"
        try:
            self.options_dic["maxalert_upper_modified"] = userOptionsDic[
                "maxalert_upper_modified"
            ]
        except:
            self.options_dic["maxalert_upper_modified"] = "-40"

        # format the Builder Dataframe for the asset builder class
        for index, row in self.spySearchResultData.iterrows():
            self.namefortag = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "DpenTag"
            ].iloc[0]
            self.pathfortag = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "taggroup"
            ].iloc[0]
            self.fullpath = "".join((f"{self.pathfortag}\\{self.namefortag}"))

            self.builderDF.at[index, "Build Asset"] = self.fullpath
            self.builderDF.at[index, "Build Path"] = self.options_dic["AssetTreeName"]
            self.builderDF.at[index, "upperbound_asset360"] = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "upperbound_asset360"
            ].iloc[0]
            self.builderDF.at[index, "lowerbound_asset360"] = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "lowerbound_asset360"
            ].iloc[0]
            # self.builderDF.at[index, 'modelname'] = modelsdf.loc[rowsdf['DpenTag'] == row['Name'], 'modelname'].iloc[0]
            self.builderDF.at[index, "modelid"] = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "modelid"
            ].iloc[0]
            self.builderDF.at[index, "InputVariable"] = ""

        # format asset path to correct seeq format
        self.builderDF["Build Asset"] = self.builderDF["Build Asset"].str.replace(
            "\\", ">>", regex=False
        )

    def tests_stagnet(self):
        try:
            self.options_dic["workbookpath"] = userOptionsDic["workbookpath"]
        except:
            self.options_dic["workbookpath"] = "MQI >> Test_stagnatinput"
        try:
            self.options_dic["AssetTreeName"] = userOptionsDic["AssetTreeName"]
        except:
            self.options_dic["AssetTreeName"] = "My Unit"
        try:
            self.options_dic["stagnant_window"] = userOptionsDic["stagnant_window"]
        except:
            self.options_dic["stagnant_window"] = "5d"
        try:
            self.options_dic["stagnant_req"] = userOptionsDic["stagnant_req"]
        except:
            self.options_dic["stagnant_req"] = "5d"
        try:
            self.options_dic["stagnant_reqalerttime"] = userOptionsDic[
                "stagnant_reqalerttime"
            ]
        except:
            self.options_dic["stagnant_reqalerttime"] = "14d"

        # format the Builder Dataframe for the asset builder class
        for index, row in self.spySearchResultData.iterrows():
            self.namefortag = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "DpenTag"
            ].iloc[0]
            self.pathfortag = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "taggroup"
            ].iloc[0]
            self.fullpath = "".join((f"{self.pathfortag}\\{self.namefortag}"))

            self.builderDF.at[index, "Build Asset"] = self.fullpath
            self.builderDF.at[index, "Build Path"] = self.options_dic["AssetTreeName"]
            # self.builderDF.at[index, 'modelname'] = modelsdf.loc[rowsdf['DpenTag'] == row['Name'], 'modelname'].iloc[0]
            self.builderDF.at[index, "modelid"] = self.rowsdf.loc[
                self.rowsdf["DpenTag"] == row["Name"], "modelid"
            ].iloc[0]
            self.builderDF.at[index, "InputVariable"] = row["Name"]

        # format asset path to correct seeq format
        self.builderDF["Build Asset"] = self.builderDF["Build Asset"].str.replace(
            "\\", ">>", regex=False
        )


In [ ]:
def assetbuilderfunc_zscore(signal_meta, class_tag):
    class Asset_class(Asset):

        # Main function for signals
        @Asset.Attribute()
        def main(self, metadata):

            return {
                "Name": metadata["Name"].iloc[0],
                "Type": "Signal"
                # ,'Description': metadata['Name'].Description.iloc[0]
                ,
                "Formula": "$signal",
                "Formula Parameters": {"$signal": metadata["ID"].iloc[0]},
            }

        # ********STAR:  Zscore Test: calculate Z-score test for asset 360 model bounds********
        @Asset.Attribute()
        def std_dev_signal(self, metadata):

            training_window = f"\n$trainingwindow = {signal_meta['sigma_window']}"
            training_freq = f"\n$trainingfreq = {signal_meta['sigma_freq']}"
            condition = (
                f"\n$trainingcondition = periods($trainingwindow, $trainingfreq)"
            )

            formulastring = " ".join(
                (
                    training_window,
                    training_freq,
                    condition,
                    f"\n$tag_sigma = $signal.aggregate(stdDev(), $trainingcondition, endKey())",
                    f"\nreturn $tag_sigma",
                )
            )

            #             descrip = (f"Asset360 Model ID: {metadata['modelid'].iloc[0]}"
            #                             f"\nAsset360 Model Name: {metadata['modelname'].iloc[0]}")
            return {
                "Name": "Standard Deviation",
                "Type": "Signal"
                # ,'Description': descrip
                ,
                "Formula": formulastring,
                "Formula Parameters": {"$signal": self.main(metadata)},
            }

        @Asset.Attribute()
        def average_signal(self, metadata):

            training_window = f"\n$trainingwindow = {signal_meta['sigma_window']}"
            training_freq = f"\n$trainingfreq= {signal_meta['sigma_freq']}"
            condition = (
                f"\n$trainingcondition = periods($trainingwindow, $trainingfreq)"
            )
            formulastring = " ".join(
                (
                    training_window,
                    training_freq,
                    condition,
                    f"\n$tag_avg = $signal.aggregate(average(), $trainingcondition, endKey())",
                    f"\nreturn $tag_avg",
                )
            )
            return {
                "Name": "Average",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {"$signal": self.main(metadata)},
            }

        # The upper and lower bound for the model set by Asset 360
        @Asset.Attribute()
        def upperbound_asset360(self, metadata):

            return {
                "Name": "upperbound-asset360",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": metadata["upperbound_asset360"].iloc[0],
            }

        @Asset.Attribute()
        def lowerbound_asset360(self, metadata):

            return {
                "Name": "lowerbound-asset360",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": metadata["lowerbound_asset360"].iloc[0],
            }

        # caluclated Z score for upper and lower bounds
        @Asset.Attribute()
        def upper_zscore_calc(self, metadata):

            formulastring = " ".join(
                (
                    f"\n$upper_zscore_calc = ($upperbound_asset360-$average)/$standardDev",
                    f"\nreturn $upper_zscore_calc",
                )
            )

            return {
                "Name": "Upper Bound Z-Score",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {
                    "$average": self.average_signal(metadata),
                    "$standardDev": self.std_dev_signal(metadata),
                    "$upperbound_asset360": self.upperbound_asset360(metadata),
                },
            }

        @Asset.Attribute()
        def lower_zscore_calc(self, metadata):

            formulastring = " ".join(
                (
                    f"\n$lower_zscore_calc = ($lowerbound_asset360-$average)/$standardDev",
                    f"\nreturn $lower_zscore_calc",
                )
            )
            return {
                "Name": "lower Bound Z-Score",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {
                    "$average": self.average_signal(metadata),
                    "$standardDev": self.std_dev_signal(metadata),
                    "$lowerbound_asset360": self.lowerbound_asset360(metadata),
                },
            }

        @Asset.Attribute()
        def upper_modified_zscore_calc(self, metadata):

            formulastring = " ".join(
                (
                    f"\n$upper_mod_zscore = ($upperbound_asset360-$average)/($average*0.30)",
                    f"\nreturn $upper_mod_zscore",
                )
            )

            return {
                "Name": "Upper Bound Modified Z-Score",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {
                    "$average": self.average_signal(metadata),
                    "$standardDev": self.std_dev_signal(metadata),
                    "$upperbound_asset360": self.upperbound_asset360(metadata),
                },
            }

        @Asset.Attribute()
        def lower_modified_zscore_calc(self, metadata):

            formulastring = " ".join(
                (
                    f"\n$lower_mod_zscore = ($lowerbound_asset360-$average)/($average*0.30)",
                    f"\nreturn $lower_mod_zscore",
                )
            )
            return {
                "Name": "lower Bound Modified Z-Score",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {
                    "$average": self.average_signal(metadata),
                    "$standardDev": self.std_dev_signal(metadata),
                    "$lowerbound_asset360": self.lowerbound_asset360(metadata),
                },
            }

        @Asset.Attribute()
        def combined_zscore_calc(self, metadata):

            formulastring = " ".join(
                (
                    f"\n$combined_zscore_val = abs($lower_zscore_calc) + abs($upper_zscore_calc)",
                    f"\nreturn $combined_zscore_val",
                )
            )
            return {
                "Name": "Combined Bound Z-Score",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {
                    "$lower_zscore_calc": self.lower_zscore_calc(metadata),
                    "$upper_zscore_calc": self.upper_zscore_calc(metadata),
                },
            }

        # The max limits for upper and lower z score- used as bound for capsule to alert on
        @Asset.Attribute()
        def max_upperalertlimit(self, metadata):

            return {
                "Name": "Max Upper Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_upper"],
            }

        @Asset.Attribute()
        def max_loweralertlimit(self, metadata):

            return {
                "Name": "Max Lower Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_lower"],
            }

        @Asset.Attribute()
        def max_combined_upperalertlimit(self, metadata):

            return {
                "Name": "Max combined Upper Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_upper_combined"],
            }

        @Asset.Attribute()
        def max_combined_loweralertlimit(self, metadata):

            return {
                "Name": "Max combined Lower Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_lower_combined"],
            }

        @Asset.Attribute()
        def max_modified_upperalertlimit(self, metadata):

            return {
                "Name": "Max Modified Upper Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_upper_modified"],
            }

        @Asset.Attribute()
        def max_modified_loweralertlimit(self, metadata):

            return {
                "Name": "Max Modified Lower Alert Limit",
                "Type": "Scalar"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": signal_meta["maxalert_lower_modified"],
            }

        # The capsule that alerts using the bounds set in another function
        @Asset.Attribute()
        def upperbound_ZScoreLimit(self, metadata):

            formulastring = f"($upper_zscore_calc > $max_upperalertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Upper Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$upper_zscore_calc": self.upper_zscore_calc(metadata),
                    "$max_upperalertlimit": self.max_upperalertlimit(metadata),
                },
            }

        @Asset.Attribute()
        def lowerbound_ZScoreLimit(self, metadata):

            formulastring = f"($lower_zscore_calc < $max_loweralertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Lower Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$lower_zscore_calc": self.lower_zscore_calc(metadata),
                    "$max_loweralertlimit": self.max_loweralertlimit(metadata),
                },
            }

        @Asset.Attribute()
        def upper_combined_condition_ZScore(self, metadata):

            formulastring = f"($combined_zscore_calc > $max_combined_upperalertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Upper Combined Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$combined_zscore_calc": self.combined_zscore_calc(metadata),
                    "$max_combined_upperalertlimit": self.max_combined_upperalertlimit(
                        metadata
                    ),
                },
            }

        @Asset.Attribute()
        def lower_combined_condition_ZScore(self, metadata):

            formulastring = f"($combined_zscore_calc < $max_combined_loweralertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Lower Combined Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$combined_zscore_calc": self.combined_zscore_calc(metadata),
                    "$max_combined_loweralertlimit": self.max_combined_loweralertlimit(
                        metadata
                    ),
                },
            }

        @Asset.Attribute()
        def upper_modified_condition_ZScore(self, metadata):

            formulastring = f"($upper_modified_zscore_calc > $max_modified_upperalertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Upper modified Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$upper_modified_zscore_calc": self.upper_modified_zscore_calc(
                        metadata
                    ),
                    "$max_modified_upperalertlimit": self.max_modified_upperalertlimit(
                        metadata
                    ),
                },
            }

        @Asset.Attribute()
        def lower_modified_condition_ZScore(self, metadata):

            formulastring = f"($lower_modified_zscore_calc < $max_modified_loweralertlimit).removeShorterThan({signal_meta['zscore_reqalerttime']})"

            return {
                "Name": "Lower modified Condition Limit",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$lower_modified_zscore_calc": self.lower_modified_zscore_calc(
                        metadata
                    ),
                    "$max_modified_loweralertlimit": self.max_modified_loweralertlimit(
                        metadata
                    ),
                },
            }

        # ********END:  Zscore Test: calculate Z-score test for asset 360 model bounds********

        # ********START: stagnant variable test: test if singal does not change********
        @Asset.Attribute()
        def stagnantDpenVar_sig(self, metadata):

            training_window = f"\n$trainingwindow = {signal_meta['stagnant_window']}"
            training_freq = f"\n$trainingfreq = {signal_meta['stagnant_req']}"
            condition = (
                f"\n$trainingcondition = periods($trainingwindow, $trainingfreq)"
            )

            formulastring = " ".join(
                (
                    training_window,
                    training_freq,
                    condition,
                    f"\n$Var_Change = $signal.aggregate(delta(), $trainingcondition, endKey())",
                    f"\nreturn $Var_Change",
                )
            )

            return {
                "Name": "Stagnant Depend Variable",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {"$signal": self.main(metadata)},
            }

        # fixed limit alert for varibles not changing
        @Asset.Attribute()
        def stagnantDpenVar_FixedLimit(self, metadata):

            formulastring = f"($stagnantDpenVar_sig == 0).removeShorterThan({signal_meta['stagnant_reqalerttime']})"
            return {
                "Name": "stagnant Dependent Variable",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$stagnantDpenVar_sig": self.stagnantDpenVar_sig(metadata)
                },
            }

        # ********END: stagnant variable test: test if singal does not change********

        # set what is displayed to the user:
        @Asset.Display()
        def StandardZscore(self, metadata, analysis):
            # First we create a worksheet that is named for the Facility and Section. The analysis.worksheet()
            # function will create the worksheet on the analysis workbook if it doesn't already exist.
            worksheet = analysis.worksheet("StandardZscore")

            # Now we define a workstep with the display_items that we want on the trend.
            workstep = worksheet.workstep("StandardZscore")
            workstep.display_items = [
                {"Item": self.upper_zscore_calc(), "Lane": 1, "Axis Group": "A"},
                {"Item": self.lower_zscore_calc(), "Lane": 1, "Axis Group": "A"},
                {"Item": self.max_upperalertlimit(), "Lane": 1, "Axis Group": "A"},
                {"Item": self.max_loweralertlimit(), "Lane": 1, "Axis Group": "A"},
                {"Item": self.upperbound_ZScoreLimit(), "Lane": 1, "Axis Group": "A"},
                {"Item": self.lowerbound_ZScoreLimit(), "Lane": 1, "Axis Group": "A"},
            ]

            return workstep

        # set what is displayed to the user:
        @Asset.Display()
        def combinedZscore(self, metadata, analysis):
            # First we create a worksheet that is named for the Facility and Section. The analysis.worksheet()
            # function will create the worksheet on the analysis workbook if it doesn't already exist.
            worksheet = analysis.worksheet("combinedZscore")

            # Now we define a workstep with the display_items that we want on the trend.
            workstep = worksheet.workstep("combinedZscore")
            workstep.display_items = [
                {"Item": self.combined_zscore_calc(), "Lane": 1, "Axis Group": "A"},
                {
                    "Item": self.max_combined_upperalertlimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.max_combined_loweralertlimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.upper_combined_condition_ZScore(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.lower_combined_condition_ZScore(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
            ]

            return workstep

        # set what is displayed to the user:
        @Asset.Display()
        def modifiedZscore(self, metadata, analysis):
            # First we create a worksheet that is named for the Facility and Section. The analysis.worksheet()
            # function will create the worksheet on the analysis workbook if it doesn't already exist.
            worksheet = analysis.worksheet("modifiedZscore")

            # Now we define a workstep with the display_items that we want on the trend.
            workstep = worksheet.workstep("modifiedZscore")
            workstep.display_items = [
                {
                    "Item": self.upper_modified_zscore_calc(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.lower_modified_zscore_calc(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.max_modified_upperalertlimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.max_modified_loweralertlimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.upper_modified_condition_ZScore(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
                {
                    "Item": self.lower_modified_condition_ZScore(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
            ]

            return workstep

        # set what is displayed to the user:
        @Asset.Display()
        def stagnantDpenVar(self, metadata, analysis):
            # First we create a worksheet that is named for the Facility and Section. The analysis.worksheet()
            # function will create the worksheet on the analysis workbook if it doesn't already exist.
            worksheet = analysis.worksheet("stagnantDpenVar")

            # Now we define a workstep with the display_items that we want on the trend.
            workstep = worksheet.workstep("stagnantDpenVar")
            workstep.display_items = [
                {"Item": self.stagnantDpenVar_sig(), "Lane": 1, "Axis Group": "A"},
                {
                    "Item": self.stagnantDpenVar_FixedLimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
            ]

            return workstep

    Asset_class.__qualname__ = class_tag

    return Asset_class


In [ ]:
def assetbuilderfunc_stagnat(signal_meta, class_tag):
    class InputVariable_Class(Asset):
        # Main function for signals
        @Asset.Attribute()
        def main(self, metadata):

            return {
                "Name": metadata["Name"].iloc[0],
                "Type": "Signal"
                # ,'Description': metadata['Name'].Description.iloc[0]
                ,
                "Formula": "$signal",
                "Formula Parameters": {"$signal": metadata["ID"].iloc[0]},
            }

        @Asset.Attribute()
        def stagnantIndVar_sig(self, metadata):

            training_window = f"\n$trainingwindow = {signal_meta['stagnant_window']}"
            training_freq = f"\n$trainingfreq = {signal_meta['stagnant_req']}"
            condition = (
                f"\n$trainingcondition = periods($trainingwindow, $trainingfreq)"
            )

            formulastring = " ".join(
                (
                    training_window,
                    training_freq,
                    condition,
                    f"\n$Var_Change = $signal.aggregate(delta(), $trainingcondition, endKey())",
                    f"\nreturn $Var_Change",
                )
            )

            return {
                "Name": "Stagnant Indipendent Variable",
                "Type": "Signal"
                # ,'Description': metadata['Description'].Description.iloc[0]
                ,
                "Formula": formulastring,
                "Formula Parameters": {"$signal": self.main(metadata)},
            }

        # fixed limit alert for varibles not changing
        @Asset.Attribute()
        def stagnantIndVar_FixedLimit(self, metadata):

            formulastring = f"($stagnantindVar_sig == 0).removeShorterThan({signal_meta['stagnant_reqalerttime']})"
            return {
                "Name": "stagnant Independent Variable",
                "Type": "Condition",
                "Formula": formulastring,
                "Formula Parameters": {
                    "$stagnantindVar_sig": self.stagnantIndVar_sig(metadata)
                },
            }
            # set what is displayed to the user:

        @Asset.Display()
        def stagnantDpenVar(self, metadata, analysis):
            # First we create a worksheet that is named for the Facility and Section. The analysis.worksheet()
            # function will create the worksheet on the analysis workbook if it doesn't already exist.
            worksheet = analysis.worksheet("stagnantDpenVar")

            # Now we define a workstep with the display_items that we want on the trend.
            workstep = worksheet.workstep("stagnantDpenVar")
            workstep.display_items = [
                {"Item": self.stagnantIndVar_sig(), "Lane": 1, "Axis Group": "A"},
                {
                    "Item": self.stagnantIndVar_FixedLimit(),
                    "Lane": 1,
                    "Axis Group": "A",
                },
            ]

    InputVariable_Class.__qualname__ = class_tag

    return InputVariable_Class
